# **Final Data Prep.**:<a class="anchor" id="0"></a>

1. [**Data Import**](#1)
2. [**Null Imputation**](#2)
3. [**Data Merge**](#3)
4. [**Pickling**](#4)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from _util.custom_mem_opt import custom_mem_opt

from pprint import PrettyPrinter

pp = PrettyPrinter(width=41, compact=True)
pd.options.mode.chained_assignment = None
pd.set_option('future.no_silent_downcasting', True)

### Data Import <a class=anchor id="1"></a>
[Back to top](#0)

Apply memory optimization.

In [3]:
root = './_data/'

aisles = custom_mem_opt(pd.read_csv(root + 'aisles.csv'), verbose=False)
departments = custom_mem_opt(pd.read_csv(root + 'departments.csv'), verbose=False)
orders = custom_mem_opt(pd.read_csv(root + 'orders.csv'), verbose=False)
order_products_prior = custom_mem_opt(pd.read_csv(root + 'order_products__prior.csv'), verbose=False)
order_products_train = custom_mem_opt(pd.read_csv(root + 'order_products__train.csv'), verbose=False)
products = custom_mem_opt(pd.read_csv(root + 'products.csv'), verbose=False)

In [4]:
root = './_pkls/'

cust_features = pd.read_pickle(root + 'cust_features.p')
cust_product_features = pd.read_pickle(root + 'cust_product_features.p')
product_features = pd.read_pickle(root + 'product_features.p')

In [5]:
train_orders = orders.merge(order_products_train, on = 'order_id', how = 'inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [6]:
train_orders.drop(['eval_set', 'add_to_cart_order', 'order_id'], axis = 1, inplace = True)

In [7]:
train_users = train_orders.user_id.unique()
pp.pprint(train_users[:10])

array([ 1,  2,  5,  7,  8,  9, 10, 13, 14, 17], dtype=int32)


In [8]:
print(cust_product_features.shape)

(13307953, 11)
